# Vežbe iz konvolucionih neuralnih mreža - Uvod
## Letnji kamp za nove polaznike

Cilj ove vežbe je da pruži prvi kontakt sa konvolucionim neuralnim mrežama, ali i da vas upozna sa bibliotekom [PyTorch](https://pytorch.org).

PyTorch je biblioteka koja automatizuje jako puno procesa koji bi se inače morali ručno pisati. Glavni od njih se tiče __komputacionog grafa__ i algoritma __propagacije greške unazad__. Naime, PyTorch implementira sve savremene slojeve neuralnih mreža, i nudi mogućnost njihovog povezivanja u neuralne mreže. Posledično, PyTorch modeli automatski održavaju komputacioni graf i računaju sve potrebne gradijente!

Ova sveska je okvirno podeljena na sledeće celine:

* Baratanje podacima
* Kreiranje neuralne mreže
* Treniranje neuralne mreže

## Baratanje podacima

Upoznavanje sa PyTorch-om i konvolucionim neuralnim mrežama ćemo odraditi nad zadatkom klasifikacije odeće iz dataseta `FashionMNIST`. Iako se informacije o ovom skupu podataka mogu naći vrlo jednostavno na internetu, počnimo od toga da ne znamo ništa o njemu i da moramo da ga malo istražimo.

Za početak učitajmo potrebne biblioteke:

In [ ]:
import numpy as np
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

from utils import sample_to_pil_image

U ćeliji ispod učitavamo `FashionMNIST` skup podataka. [PyTorch Datsets](https://pytorch.org/vision/stable/datasets.html) je bogata biblioteka sa puno skupova podataka koji se koriste za različite namene. Prednost korišćenja tih skupova podataka je što se učitavaju u doslovno četiri linije koda, kao što je prikazano ispod. Mnogi skupovi podataka, međutim, imaju određene specifičnosti i ne podpadaju pod datu biblioteku, kao što ćete videti u narednim notebook-ovima.

In [ ]:
train_set = torchvision.datasets.FashionMNIST("../dataset/data", download=True, train=True, transform=transforms.Compose([transforms.ToTensor()]))
test_set = torchvision.datasets.FashionMNIST("../dataset/data", download=True, train=False, transform=transforms.Compose([transforms.ToTensor()]))

train_loader = torch.utils.data.DataLoader(train_set, shuffle=False, num_workers=2, batch_size=100)
test_loader = torch.utils.data.DataLoader(test_set, shuffle=False, num_workers=2, batch_size=100)

Pre svega pogledajmo kako izgleda naša baza slika. Slobodno pokrenite ćeliju ispod više puta kako biste videli različite primere / klase.

In [ ]:
def get_random_sample(dataloader):
    batched_images, batched_classes = next(iter(dataloader))
    random_index = np.random.randint(0, batched_images.shape[0])
    random_image, random_class = batched_images[random_index], batched_classes[random_index]
    return random_image, random_class

random_image, random_label = get_random_sample(dataloader=test_loader)
image = sample_to_pil_image(sample=random_image, image_shape=random_image.shape[1:])
print(f"Sample image for class {test_set.classes[random_label]} is shown below:")
display(image)

Hajde kvantitativno da ispitamo dataset. Kod ispod najpre proverava koliko imamo primera u skupu za obučavanje i skupu za testiranje, a posle proveri koliko imamo primera po klasi. Skpovi podataka koji imaju otprilike jednak broj primera za svaku klasu nazivaju se __balansiranim__ skupovima podataka i sa njima je mnogo lakše raditi:

1. __tačnost__ kao metrika ima smisla. Posledično ne moramo uvoditi komplikovanije metrike
2. Ne moramo posebno da pazimo da raspodela klasa u skupu za obučavanje i testiranje budu iste (nasumično uzorkovanje će to obezbediti)
3. Ne moramo uvoditi augmentacije podataka, veštački balansirati skup podataka, i sl.
4. Mreža prirodno ima priliku da se obuči podjednako dobro za svaku klasu (naravno neke mogu biti teže, ali makar ih ima u jednakoj meri)

Kao što vidimo, na svu sreću, `FashionMNIST` je balansiran.

In [ ]:
from collections import defaultdict

print(f"Train set consists of {len(train_set)} categorized into {len(train_set.classes)} classes.")
print(f"Test set consists of {len(test_set)} categorized into {len(test_set.classes)} classes.")
print(f"Image dimension is {random_image.shape}")

label_dict = defaultdict(int)
for _, label in train_set:
    label_name = train_set.classes[label]
    label_dict[label_name] += 1
print(label_dict)

label_dict = defaultdict(int)
for _, label in test_set:
    label_name = test_set.classes[label]
    label_dict[label_name] += 1
print(label_dict)

## Kreiranje mreže

Na vežbama iz neuralnih mreža smo morali za svaki sloj ručno da implementiramo:
* Prolazak podataka unapred kroz `forward` metodu
* Propagaciju greške unazad kroz `backward` metodu

PyTorch nudi dve opcije:
* Korišćenje već gotovog sloja kome su obe metode implementirane
* Definisanje svog sloja __kome je potrebno implementirati samo `forward` metodu!__ Na osnovu operacija u toj metodi, i po pravilu propagacije greške unazad, PyTorch sam računa gradijente

U ćeliji ispod koristimo prvi pristup, gde su nam svi slojevi i operacije već implementirane. Konkretno, prvi sloj mreže čine:

1. Krećemo od operacije konvolucije `Conv2d` sa brojem neurona 32, veličinom filtra 3 i popunjavanjem ivica (__padding__) 1.
2. Normalizujemo izlaz prethodne operacije `BatchNorm2d` operacijom
3. Primenimo aktivacionu funkciju `ReLU`
4. Primenimo `MaxPool2d` sa veličinom filtra 2 i korakom 2

Možda je malo konfuzno, ali većina od prethodno navedenih operacija se sama po sebi može nazvati slojem, a možemo ih grupisati i celu sekvencu operacija takođe nazvati slojem! Konkretno, PyTorch koristi klasu [Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) da predstavi sloj, pri čemu `Module` može sadržati druge `Module`-e.

#### Pitanja

1. Zašto je `in_channels=1`?
1. Zašto nam je `out_features=10` u poslednjem?

In [ ]:
mnist_architecture = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    
    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),

    nn.Linear(in_features=64*6*6, out_features=600),
    nn.Dropout(0.25),
    nn.Linear(in_features=600, out_features=120),
    nn.Linear(in_features=120, out_features=10)
)
neural_net = mnist_architecture

Konvolucione neuralne mreže znaju biti poprilično duboke i imati jako puno slojeva. Treniranje takvih mreža je najčešće vrlo komputaciono zahtevno, te je bitno na kom uređaju se treniraju:

* CPU (procesor): Manji potencijal za paralelizacijom, ali brza jezgra.
* GPU (grafička kartica): Veliki potencijal za paralelizacijom, ali spora jezgra.

Zbog prirode operacija neuralnih mreža koje se mogu paralelizovati, najviše im odgovara treniranje na grafičkoj kartici. PyTorch nudi vrlo jednostavno prebacivanje podataka sa jednog resursa na drugi.

Ćelija ispod napre odredi koji tip resursa nam je dostupan i odabere GPU ako postoji, i onda prebaci model na odgovarajući resurs.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
neural_net.to(device)

## Treniranje neuralne mreže

U ćeliji ispod definišemo:
1. __funkciju gubitka__ koju želimo da koristimo, koja je ista kao i na prethodnoj vežbi. 
2. stopu učenja
3. __optimizator__: Ovo je novi koncept. Na prošlim vežbama je bilo reči o algoritmu __gradijentnog spusta__. Postoje tri tipa gradijentnog spusta:
    * __stohastički gradijentni spust__: Osvežavanje težina mreže se radi nakon svakog primera koji prođe kroz mrežu, odnosno svaki primer zasebno utiče na težine.
    * __šaržni gradijentni spust__: Osvežavanje težina mreže se radi tek nakon što svi primeri prođu kroz mrežu, odnosno gradijenti greške za svaki od njih se najpre usrednje.
    * __mini-šaržni gradijentni spust__: Između stohastičkog i šaržnog, primeri prolaze kroz mrežu u grupama fiksne veličine.
    
    Svaki od ovih tipova gradijentnog spusta, međutim, se može koristiti sa još puno dodatnih parametara koji mogu značajno uticati na brzinu konvergencije modela. Zbog toga PyTorch uvodi klasu `Optimizer` koja enkapsulira parametre i tip gradijentnog spusta.

#### Pitanja

1. Koji tip gradijentnog spusta deluje da se koristi ispod? Ako pogledamo deo koda gde kreiramo `DataLoader`, da li deluje da koristimo mini-šaržni gradijentni spust?
    

In [ ]:
NUM_EPOCHS = 5

loss_function = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.SGD(neural_net.parameters(), lr=learning_rate)

Kod ispod trenira neuralnu mrežu. Komentari u kodu objašnjavaju i specifičnosti PyTorch-a, ali i povezuju treniranje sa prethodnim vežbama.

In [ ]:
from utils import get_accuracy, plot_curve, keep_store_dict, store_dict_to_disk

def test(model, test_loader):
    # Put the model in evaluation mode. 
    # Tells the model not to compute gradients. Increases inference speed.
    model.eval()
    test_acc = 0.0
    with torch.no_grad():
        for batch_num, (x, y) in enumerate(test_loader, 0):
            # Put the data to the appropriate device.
            x = x.to(device)
            y = y.to(device)
            # Do inference. Forwad pass with the model.
            y_hat = model(x)
            test_acc += get_accuracy(y_hat, y)
    return test_acc / batch_num

def train(model, num_epochs, train_loader, store_dict, test_loader=None):
    for epoch in range(num_epochs):
        # Keep track of accuracy and loss accross batches
        train_running_loss = 0.0
        train_acc = 0.0

        # Put the model in training mode. 
        # Tells the model to keep track of gradients. Reduces inference speed.
        model = model.train()

        # A single training step
        for batch_num, (x, y) in enumerate(train_loader):
            
            # Put the data to the appropriate device.
            x = x.to(device)
            y = y.to(device)

            # Do inference. Forwad pass with the model.
            y_hat = model(x)
            
            # Calculate the loss. Recall the computation graph.
            loss = loss_function(y_hat, y)
            
            # Do backpropagation and gradient descent step. ####
            # 1. Reset remembered gradients. (Optimizers remember gradients from previous 
            # iterations, we do not want these to affect the current step)
            optimizer.zero_grad()
            # Do backpropagation algorithm and calculate all relevant gradients.
            loss.backward()
            # Update model parameters (weights and biases) w.r.t. computed gradients.
            optimizer.step()

            train_running_loss += loss.detach().item()
            train_acc += get_accuracy(y_hat=y_hat, y=y)

        
        epoch_loss = train_running_loss / batch_num
        epoch_acc = train_acc / batch_num
        
        store_dict = keep_store_dict(curve=epoch_loss, label='train_loss', store_dict=store_dict)
        store_dict = keep_store_dict(curve=epoch_acc, label='train_acc', store_dict=store_dict)
        print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' \
            %(epoch, epoch_loss, epoch_acc))

        if test_loader is not None:
            test_acc = test(model=model, test_loader=test_loader)
            store_dict = keep_store_dict(curve=test_acc, label='test_acc', store_dict=store_dict)
        
    return store_dict

Ćelija ispod pušta trening i čuva rezultate na disk, na putanju `results/mnist.json`.

__Napomena__: Ćelija se može puštati više puta u kom slučaju će trening modela biti nastavljen. Rezultati koji se čuvaju na disku će biti samo nadovezani na već postojeće. Ukoliko želite da trenirate model od nule, iznova, potrebno je pustiti sve ćelije od definicije modela pa na dalje, i __ručno izbrisati `results/mnist.json`__.

In [ ]:
mnist_result_path = 'results/mnist.json'
store_dict = train(model=neural_net, num_epochs=NUM_EPOCHS, train_loader=train_loader, test_loader=test_loader, store_dict=None)
store_dict_to_disk(file_path=mnist_result_path, store_dict=store_dict)

Rezultati su sačuvani na putanji `results/mnist.json`. Rezultate sada možemo plotovati puštanjem ćelije ispod.

### Pitanja

1. Da li je model dovoljno obučen?
2. Da li je model preobučen?

In [ ]:
# Plot results
from utils import load_dict_from_disk

store_dict = load_dict_from_disk(file_path=mnist_result_path)

# We plot accuracy here. We could also have plotted the loss.
fig = plot_curve(curves=[store_dict['train_acc'], store_dict['test_acc']], labels=['train_acc', 'test_acc'], plot_name='accuracy')

Konačno, u sledećoj ćelji možemo videti kako model zaključuje na nasumičnim primerima.

In [ ]:
# If we have not already loaded sample image in the intro
if random_image is None:
    random_image, random_label = get_random_sample(dataloader=test_loader)

image = sample_to_pil_image(sample=random_image, image_shape=random_image.shape[1:])

x = torch.unsqueeze(random_image.to(device), 0)
y_hat = np.argmax(neural_net(x).detach().cpu().numpy())
y_hat = test_set.classes[y_hat]
y = test_set.classes[random_label]
print(y_hat)

print(f"Sample image for class {y} is shown below. Model prediction is {y_hat}.")
display(image)

random_image, random_label = get_random_sample(dataloader=test_loader)